In [4]:
pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import psycopg2
import boto3
import pandas as pd
from io import StringIO

# Redshift Serverless connection info
REDSHIFT_HOST = "wg1.250260913801.ap-south-1.redshift-serverless.amazonaws.com"
REDSHIFT_PORT = 5439
REDSHIFT_DB = "dev"
REDSHIFT_USER = "admin"
REDSHIFT_PASS = "Admin-123"

# AWS S3 details
S3_BUCKET = "sales-leaddd"
S3_KEY = "redshift-exports/data.csv"
AWS_REGION = "ap-south-1"

# 1. Connect to Redshift Serverless
conn = psycopg2.connect(
    host=REDSHIFT_HOST,
    port=REDSHIFT_PORT,
    dbname=REDSHIFT_DB,
    user=REDSHIFT_USER,
    password=REDSHIFT_PASS,
    sslmode='require'  # since SSL is enabled on your serverless instance
)
print('connected')

# 2. Query your data from Redshift Serverless
query = "SELECT 1 AS dummy_col;"
df = pd.read_sql(query, conn)
print(df)

conn.close()

# 3. Convert dataframe to CSV in memory
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)

# 4. Upload CSV data to S3 bucket
s3_client = boto3.client('s3', region_name=AWS_REGION)
s3_client.put_object(Bucket=S3_BUCKET, Key=S3_KEY, Body=csv_buffer.getvalue())

print(f"Data uploaded to s3://{S3_BUCKET}/{S3_KEY}")


connected
   dummy_col
0          1


/tmp/ipykernel_1231/3415074393.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


ClientError: An error occurred (AccessDenied) when calling the PutObject operation: User: arn:aws:sts::250260913801:assumed-role/AmazonSageMaker-ExecutionRole-20250718T130675/SageMaker is not authorized to perform: s3:PutObject on resource: "arn:aws:s3:::sales-leaddd/redshift-exports/data.csv" because no identity-based policy allows the s3:PutObject action

In [3]:
import psycopg2
import pandas as pd

# Redshift connection info
REDSHIFT_HOST = "wg1.250260913801.ap-south-1.redshift-serverless.amazonaws.com"
REDSHIFT_PORT = 5439
REDSHIFT_DB = "dev"
REDSHIFT_USER = "admin"
REDSHIFT_PASS = "Admin-123"

# Your target table name
TABLE_NAME = "leads_dataa"  # <-- Replace this with your actual table name

# Connect to Redshift
conn = psycopg2.connect(
    host=REDSHIFT_HOST,
    port=REDSHIFT_PORT,
    dbname=REDSHIFT_DB,
    user=REDSHIFT_USER,
    password=REDSHIFT_PASS,
    sslmode='require'
)

# Read entire table into pandas dataframe
query = f"SELECT * FROM {TABLE_NAME};"
df = pd.read_sql(query, conn)
conn.close()

# Save dataframe to CSV locally
csv_file_path = f"{TABLE_NAME}.csv"
df.to_csv(csv_file_path, index=False)
print(f"Entire table exported to CSV file: {csv_file_path}")

# (Optional) Preview the data
print(df.head())


Entire table exported to CSV file: leads_dataa.csv
                            prospect id  lead number              lead origin  \
0  7927b2df-8bba-4d29-b9a2-b6e0beafe620       660737                      API   
1  2a272436-5132-4136-86fa-dcc88c88f482       660728                      API   
2  8cc8c611-a219-4f35-ad23-fdfd2656bd8a       660727  Landing Page Submission   
3  0cc2df48-7cf4-4e39-9de9-19797f9b38cc       660719  Landing Page Submission   
4  3256f628-e534-4826-9d63-4a8b88782852       660681  Landing Page Submission   

      lead source do not email do not call  converted  totalvisits  \
0      Olark Chat           No          No          0            0   
1  Organic Search           No          No          0            5   
2  Direct Traffic           No          No          1            2   
3  Direct Traffic           No          No          0            1   
4          Google           No          No          1            2   

   total time spent on website  page view

/tmp/ipykernel_1231/3470157431.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
